In [ ]:
!pip install spektral

In [4]:
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout, Input
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

from spektral.data.loaders import SingleLoader


import numpy as np
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout, Input
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

from spektral.data.loaders import SingleLoader
from spektral.datasets.citation import Citation
from spektral.layers import ChebConv
from spektral.transforms import AdjToSpTensor, LayerPreprocess
from spektral.layers import ChebConv
from spektral.transforms import AdjToSpTensor, LayerPreprocess

In [5]:
dataset = 'cora'
A, X, y, train_mask, val_mask, test_mask =Citation("cora", transforms=[LayerPreprocess(ChebConv), AdjToSpTensor()])




# Parameters
channels = 100          # Number of channels in the first layer
K = 3                   # Max degree of the Chebyshev polynomials
N = X.shape[0]          # Number of nodes in the graph
F = X.shape[1]          # Original size of node features
n_classes = y.shape[1]  # Number of classes
dropout = 0.5           # Dropout rate for the features=
l2_reg = 5e-4/2       # L2 regularization rate
learning_rate = 1e-2    # Learning rate
epochs = 200            # Number of training epochs
es_patience = 10        # Patience for early stopping

fltr = ChebConv.preprocess(A).astype('f4')
X = X.toarray()

/usr/local/lib/python3.6/dist-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


ValueError: ignored

In [ ]:
X_in = Input(shape=(F, ))
fltr_in = Input((N, ), sparse=True)

dropout_1 = Dropout(dropout)(X_in)
graph_conv_1 = ChebConv(channels,
                        K=K,
                        activation='relu',
                        kernel_regularizer=l2(l2_reg),
                        use_bias=False)([dropout_1, fltr_in])
dropout_2 = Dropout(dropout)(graph_conv_1)
graph_conv_2 = ChebConv(n_classes,
                        K=K,
                        activation='softmax',
                        use_bias=False)([dropout_2, fltr_in])



model = Model(inputs=[X_in, fltr_in], outputs=graph_conv_2)
optimizer = Adam(lr=learning_rate)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              weighted_metrics=['acc'])
#model.summary()

In [ ]:
validation_data = ([X, fltr], y, val_mask)
model.fit([X, fltr],
          y,
          sample_weight=train_mask,
          epochs=epochs,
          batch_size=N,
          validation_data=validation_data,
          shuffle=False,  # Shuffling data means shuffling the whole graph
          callbacks=[
              EarlyStopping(patience=es_patience,  restore_best_weights=True)
          ])

In [ ]:
print('Evaluating model.')
eval_results = model.evaluate([X, fltr],
                              y,
                              sample_weight=test_mask,
                              batch_size=N)
print('Done.\n'
      'Test loss: {}\n'
      'Test accuracy: {}'.format(*eval_results))

In [ ]:
Evaluating model. When K = 3 and Channel = 100
1/1 [==============================] - 0s 1ms/step - loss: 0.3234 - acc: 0.8210
Done.
Test loss: 0.32339611649513245
Test accuracy: 0.8209999799728394

In [ ]:
Evaluating model. When K = 2 and Channel = 16
1/1 [==============================] - 0s 1ms/step - loss: 0.4059 - acc: 0.7900
Done.
Test loss: 0.4059180021286011
Test accuracy: 0.7900000214576721